In [1]:
import pandas as pd
import mpu
from Bio import SeqIO

In [2]:
results = pd.read_csv('/home/tobamo/analize/project-tobamo/results/megan6_results_combined.csv', index_col=0)
nr_tax = mpu.io.read('/home/tobamo/analize/project-tobamo/analysis/results_add_nr_taxa/accession_dict.json')

In [3]:
results['nr_sseqid_key'] = results['nr_sseqid'].str.split('.').str[0]
results['nr_tax'] = results['nr_sseqid_key'].map(nr_tax).fillna('')

In [4]:
cols = ['SRR','qseqid', 'megan_tax', 'nr_tax', 'tpdb2_sseqid','nr_sseqid', 'nr_pident', 'nr_length',
       'nr_mismatch', 'nr_gapopen', 'nr_qstart', 'nr_qend', 'nr_sstart',
       'nr_send', 'nr_evalue', 'nr_bitscore',  'tpdb2_pident',
       'tpdb2_length', 'tpdb2_mismatch', 'tpdb2_gapopen', 'tpdb2_qstart',
       'tpdb2_qend', 'tpdb2_sstart', 'tpdb2_send', 'tpdb2_evalue',
       'tpdb2_bitscore', 'sequence',  'nr_sseqid_key']

In [5]:
results = results[cols]

In [6]:
results.to_csv('/home/tobamo/analize/project-tobamo/results/megan6_results_combined_add_nr_taxa.csv', index=False)

In [7]:
non_cellular = results[~results['megan_tax'].str.contains('cellular')]
cellular = results[results['megan_tax'].str.contains('cellular')]

In [8]:
results.qseqid.nunique(), non_cellular.qseqid.nunique(),cellular.qseqid.nunique()

(3406, 3186, 220)

odstrani kontrolne SRRje

In [9]:
samples_path = '/home/tobamo/analize/project-tobamo/config/samples_test.tsv'
with open(samples_path) as file: samples = [line.strip() for line in file.readlines()][1:]
test_results = results[results['SRR'].isin(samples)]

In [10]:
results.qseqid.nunique(), test_results.qseqid.nunique()

(3406, 2567)

In [11]:
test_results_non_cellular = test_results[~test_results['megan_tax'].str.contains('cellular')]
test_results_cellular = test_results[test_results['megan_tax'].str.contains('cellular')]

In [12]:
test_results.qseqid.nunique(), test_results_non_cellular.qseqid.nunique(),test_results_cellular.qseqid.nunique()

(2567, 2393, 174)

In [13]:
test_results_non_cellular.to_csv('megan6_results_combined_add_nr_taxa_non_cellular.csv')

In [14]:
fasta_in = '/home/tobamo/analize/project-tobamo/analysis/palmscan/analysis_data/test_contigs_2024-04-09.fasta'
fasta_out = '/home/tobamo/analize/project-tobamo/analysis/palmscan/analysis_data/test_contigs_2024-04-09_non_cellular.fasta'

to_remove = test_results_cellular.qseqid.unique()

# make new filtered fasta
with open(fasta_in, "r") as file, open(fasta_out, "w") as outfile:
    out_seq = (r for r in SeqIO.parse(file, "fasta") if r.id not in to_remove)
    SeqIO.write(out_seq, outfile, "fasta")